In [3]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import urllib.request
import zipfile
import os
from notebookutils import mssparkutils

StatementMeta(, 34fd193a-4ca9-4e51-9df4-b92880c8560a, 5, Finished, Available, Finished)

In [4]:
# 1. Gerar lista dos últimos 2 meses
ontem = datetime.today() - timedelta(days=1)
ultimos_meses = [(ontem - relativedelta(months=i)).strftime("%Y%m") for i in range(2)]

StatementMeta(, 34fd193a-4ca9-4e51-9df4-b92880c8560a, 6, Finished, Available, Finished)

In [5]:
# 2. Criar pasta temporária local
tmp_base = "/tmp/inf_diario"
os.makedirs(tmp_base, exist_ok=True)

StatementMeta(, 34fd193a-4ca9-4e51-9df4-b92880c8560a, 7, Finished, Available, Finished)

In [6]:
# 3. Loop para processar cada mês
for anomes in ultimos_meses:
    print(f"🔄 Processando {anomes}...")

    try:
        # 3.1. Montar URL e paths
        url = f"https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_{anomes}.zip"
        local_zip_path = f"{tmp_base}/inf_diario_fi_{anomes}.zip"
        extract_path = f"{tmp_base}/{anomes}"
        os.makedirs(extract_path, exist_ok=True)

        # 3.2. Baixar o ZIP
        urllib.request.urlretrieve(url, local_zip_path)

        # 3.3. Extrair o CSV
        with zipfile.ZipFile(local_zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)

        # 3.4. Pegar nome do CSV extraído
        csv_file = [f for f in os.listdir(extract_path) if f.endswith(".csv")][0]
        local_csv_path = os.path.join(extract_path, csv_file)

        # 3.5. Ler conteúdo e salvar no Lakehouse
        with open(local_csv_path, "r", encoding="latin1") as f:
            content = f.read()

        dest_path = f"Files/bronze/fi_ultimos2meses/inf_diario_fi_{anomes}.csv"
        mssparkutils.fs.put(dest_path, content, overwrite=True)
        print(f"✅ Sucesso ao processar {anomes}")

    except Exception as e:
        print(f"❌ Erro ao processar {anomes}: {e}")

StatementMeta(, 34fd193a-4ca9-4e51-9df4-b92880c8560a, 8, Finished, Available, Finished)

🔄 Processando 202507...
✅ Sucesso ao processar 202507
🔄 Processando 202506...
✅ Sucesso ao processar 202506
